In [ ]:
!nvcc --version
!nvidia-smi
!pip install subword-nmt
!pip install transformers

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Tue Nov 30 00:02:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |            

In [ ]:
import transformers as tf
from transformers import RobertaTokenizer,RobertaModel
import pandas as pd
import numpy as np
import torch

In [ ]:
TkSelfberta = RobertaTokenizer.from_pretrained('seyonec/PubChem10M_SMILES_BPE_450k')
Selfberta = RobertaModel.from_pretrained('seyonec/PubChem10M_SMILES_BPE_450k')
TkFastberta = RobertaTokenizer.from_pretrained('/content/drive/MyDrive/trained_models')
Fastberta = RobertaModel.from_pretrained('/content/drive/MyDrive/trained_models')

Downloading:   0%|          | 0.00/161k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/98.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/321M [00:00<?, ?B/s]

Some weights of the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


HTTPError: ignored

Organizando las variables para la funcion de la concatenacion

In [ ]:
def cat(d,p,TkSelfberta,TkFastberta,Selfberta,Fastberta):
  C = []
  cont=0
  for i in zip(d, p):
  
    inputC = TkSelfberta(i[0],return_tensors = "pt")
    inputP = TkFastberta(i[1],return_tensors = "pt")
    outputC = Selfberta(**inputC).pooler_output
    outputP = Fastberta(**inputC).pooler_output
    outputI = torch.outer(outputC[0], outputP[0])
    outputI = outputI.view(1, 768, 768)
 
    if  cont==0:
      C= outputC
      P = outputP
      I = outputI
      cont = 1
    else:
      C = torch.cat((C,outputC),0)
      P = torch.cat((P,outputP),0)
      I = torch.cat((I,outputI),0)
    
  return I

###############################################################


In [ ]:
dataFolder = '/content/drive/MyDrive/prueba/MolTrans/dataset/DAVIS'
df_train = pd.read_csv(dataFolder + '/train.csv')
df_val = pd.read_csv(dataFolder + '/val.csv')
df_test = pd.read_csv(dataFolder + '/test.csv')

#############################
d = df_test.iloc[:]['SMILES'].values.tolist()
p = df_test.iloc[:]['Target Sequence'].values.tolist()
y = df_test.Label.values
y = torch.from_numpy(y)
I = cat(d,p,TkSelfberta,TkFastberta,Selfberta,Fastberta)

In [ ]:
I.shape

torch.Size([51, 768, 768])

In [ ]:
%cd /content/drive/MyDrive/prueba/MolTrans

[Errno 2] No such file or directory: '/content/drive/MyDrive/prueba/MolTrans'
/content


###################################################################################


Aqui seria un return general para los 3 sets (return I)

In [ ]:
###########UNO
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
from torch import nn 
import copy
import os
import sys
import pdb

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, roc_curve, confusion_matrix, precision_score, recall_score, auc
from sklearn.model_selection import KFold
torch.manual_seed(1)    # reproducible torch:2 np:3
np.random.seed(1)

from config import BIN_config_DBPE
from models import BIN_Interaction_Flat
from stream import BIN_Data_Encoder
##################################
#pdb.set_trace()

validloss = [] 
testloss = []
Prediccion = []
Bin = []
logit1 = []
def texto1():
    np.savetxt("validloss.txt",validloss)
    
    #Result = np.loadtxt("") 
def texto2():
    np.savetxt("testloss.txt",testloss)
    np.savetxt("Pre.txt",Prediccion)
    np.savetxt("Bin.txt",Bin)
    np.savetxt("logit.txt",logit1)
    
def texto3():
    np.savetxt("testloss.txt",testloss)
    np.savetxt("validloss.txt",validloss)

########################################

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [ ]:
###########DOS
def test(data_generator, model):
    print("Dentro de Test")
    y_pred = []
    y_label = []
    model.eval()
    loss_accumulate = 0.0
    count = 0.0
    for i, (I,label) in enumerate(data_generator):
        print("Dentro del for")
        score = model(I.long().cuda())
        print("1")
        m = torch.nn.Sigmoid()
        logits = torch.squeeze(m(score))
        #logits = torch.squeeze((score))
        loss_fct = torch.nn.MSELoss()            
        print("2")
        label = Variable(torch.from_numpy(np.array(label)).float()).cuda()

        loss = loss_fct(logits, label)
        
        loss_accumulate += loss
        count += 1
        
        logits = logits.detach().cpu().numpy()
        #print(logits)
        
        print("3")
        label_ids = label.to('cpu').numpy()
        y_label = y_label + label_ids.flatten().tolist()
        y_pred = y_pred + logits.flatten().tolist()
        logit1.append(logits)
        print("count", count)
        print("y_label", label_ids.flatten().tolist() )
        print("y_predi", logits.flatten().tolist())
        
    loss = loss_accumulate/count
    
    fpr, tpr, thresholds = roc_curve(y_label, y_pred)
    print("4")
    precision = tpr / (tpr + fpr)

    f1 = 2 * precision * tpr / (tpr + precision + 0.00001)

    thred_optim = thresholds[5:][np.argmax(f1[5:])]

    print("optimal threshold: " + str(thred_optim))

    y_pred_s = [1 if i else 0 for i in (y_pred >= thred_optim)]
    print(y_pred_s)
    auc_k = auc(fpr, tpr)
    print("AUROC:" + str(auc_k))
    print("AUPRC: "+ str(average_precision_score(y_label, y_pred)))

    cm1 = confusion_matrix(y_label, y_pred_s)
    print('Confusion Matrix : \n', cm1)
    print('Recall : ', recall_score(y_label, y_pred_s))
    print('Precision : ', precision_score(y_label, y_pred_s))

    total1=sum(sum(cm1))
    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    print('Sensitivity : ', sensitivity1 )

    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    print('Specificity : ', specificity1)
    

    outputs = np.asarray([1 if i else 0 for i in (np.asarray(y_pred) >= 0.5)])
    print("Saliendo de Testing")
    return logits, y_pred, loss.item() #roc_auc_score(y_label, y_pred), average_precision_score(y_label, y_pred), f1_score(y_label, outputs), y_pred, loss.item()

FILE = "/content/drive/MyDrive/model/model.pth"
def main(fold_n, lr,I):
    
    config = BIN_config_DBPE()
    
    lr = lr
    BATCH_SIZE = config['batch_size']
    train_epoch = 100
    
    loss_history = []
    Pre = []
    Binario = []
    print("Test 1")
    model = BIN_Interaction_Flat(**config)
    

    print("Cargando el modelo")
    model.load_state_dict(torch.load(FILE))
    print("Modelo cargado")
    model = model.cuda()

    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model, dim = 0)
            
    opt = torch.optim.Adam(model.parameters(), lr = lr)
    #opt = torch.optim.SGD(model.parameters(), lr = lr, momentum=0.9)
    
    print('--- Data Preparation ---')
    
    params = {'batch_size': BATCH_SIZE,
              'shuffle': True,
              'num_workers': 2, 
              'drop_last': True}

    dataFolder = './dataset/DAVIS'
    df_train = pd.read_csv(dataFolder + '/train.csv')
    df_val = pd.read_csv(dataFolder + '/val.csv')
    df_test = pd.read_csv(dataFolder + '/test.csv')

    #############################
    d = df_test.iloc[:]['SMILES'].values.tolist()
    p = df_test.iloc[:]['Target Sequence'].values.tolist()
    y = df_test.Label.values
    y = torch.from_numpy(y)
    #I = cat(d,p,TkSelfberta,TkFastberta,Selfberta,Fastberta)
    ##############################
    training_set = BIN_Data_Encoder(df_test.index.values, df_test.Label.values, I)   
    training_generator = data.DataLoader(training_set, **params)

    validation_set = BIN_Data_Encoder(df_test.index.values, df_test.Label.values, I)
    validation_generator = data.DataLoader(validation_set, **params)
    
    testing_set = BIN_Data_Encoder(df_test.index.values, df_test.Label.values, I)
    testing_generator = data.DataLoader(testing_set, **params)

    #print(testing_generator)
    #for i, (d, p, d_mask, p_mask, label) in enumerate(training_generator):
     # print(model.forward(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda()))
    
    # early stopping
    max_auc = 0
    model_max = copy.deepcopy(model)

    
  

    print('--- Go for Training ---')
    torch.backends.cudnn.benchmark = True
    """
    for epo in range(train_epoch):
        
        model.train()
        for i, (d, p, d_mask, p_mask, label) in enumerate(training_generator):
            score = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda())

            label = Variable(torch.from_numpy(np.array(label)).float()).cuda()
            
            loss_fct = torch.nn.MSELoss()
            #m = torch.nn.Sigmoid()
            #n = torch.squeeze(m(score))
            n = torch.squeeze(score)
            
            loss = loss_fct(n, label)
            loss_history.append(loss)
            
            opt.zero_grad()
            loss.backward()
            opt.step()
            
            if (i % 100 == 0):
                print('Training at Epoch ' + str(epo + 1) + ' iteration ' + str(i) + ' with loss ' + str(loss.cpu().detach().numpy()))
                

            
        # every epoch test
        with torch.set_grad_enabled(False):
            #auc, auprc, f1, logits, loss = test(validation_generator, model)
            loss = test(validation_generator, model)
            #if auc > max_auc:
             #   model_max = copy.deepcopy(model)
              #  max_auc = auc
            
            print('Validation at Epoch '+ str(epo + 1) +  ' , Test loss: '+ str(loss))
            validloss.append(loss)
            texto1()
    """
    print('--- Go for Testing ---')
    try:
        with torch.set_grad_enabled(False):
            print("Entro al testing")
            #auc, auprc, f1, logits, loss = test(testing_generator, model_max)
            Binario, Pre, loss = test(testing_generator, model_max)
            print("Salio de test")
            #print('Testing AUROC: ' + str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: '+str(f1) + ' , Test loss: '+str(loss))
            print( ' Test loss: '+str(loss))
            print("Guardando en la lista")
            ##################################
            Prediccion.append(Pre)
            testloss.append(loss)
            Bin.append(Binario)
            texto2()
            #torch.save(model.state_dict(), FILE)
    except Exception as e:
        #print('testing failed')
        print('testing failed: {}'.format(e))
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        texto3()
    return model_max, loss_history


In [ ]:
s = time()
model_max, loss_history = main(1, 5e-6,I)
e = time()
print(e-s)
lh = list(filter(lambda x: x < 1, loss_history))
plt.plot(lh)

Test 1
Cargando el modelo
Modelo cargado
--- Data Preparation ---
--- Go for Training ---
--- Go for Testing ---
Entro al testing
Dentro de Test
Dentro del for


In [ ]:
from torch.utils import data
testing_set = BIN_Data_Encoder(df_test.index.values, df_test.Label.values, I)
generator = data.DataLoader((testing_set),10)

NameError: ignored

Moltrans

In [ ]:
for i, (I,y) in enumerate(generator):
  print("I")
  print(I.shape)
  print("y")
  print(y.shape)

NameError: ignored